## Imports

In [1]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype

## Functions

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/Research/')

In [3]:
import functions_

In [4]:
from functions_ import getPivotMonths
from functions_ import Time_DataSplit
from functions_ import TestTrain_DataSplit
from functions_ import SingleRatingMatrix
from functions_ import AllRatingMatrices
from functions_ import All_SingleStepRatMat
from functions_ import psiStep_RatMat
from functions_ import psiAllStep_RatMat
from functions_ import Find_NewUsersItems
from functions_ import integrator
from functions_ import getStartingValues
from functions_ import integratorOnMat
from functions_ import Find_NewUsersItems
from functions_ import last_psiTrainMat
from functions_ import Updt_RowMatrix
from functions_ import getRow_Mat
from functions_ import row_update
from functions_ import Updt_ColMatrix
from functions_ import getCol_Mat
from functions_ import colunm_update
from functions_ import UsersItems_RatPair
from functions_ import getRowCol_psiupdt
from functions_ import ITEMS_defferredStatus
from functions_ import USERS_defferredStatus
from functions_ import V_listUpdate
from functions_ import SingleStep_UPDATE
from functions_ import ALLSTEPs_update
from functions_ import get_NEWHoldout
from functions_ import adjustedAllDF
from functions_ import adjustedPSI_DF
from functions_ import adjustedAllDF
from functions_ import getALLTopNPred
from functions_ import TQDMgetALLTopNPred
from functions_ import getALLTopNPred_ALLUSERS
from functions_ import TQDMgetALLTopNPred_ALLUSERS
from functions_ import getAll_HitRate
from functions_ import get_ALLRandPred,getAll_RandomHitRate,getAll_MOSTPOP_Pred,getAll_MostPOPHitRate


## Apply Functions on *AmazonBeauty*

###Step 1: Data Cleaning

In [ ]:
BeautyDFF = BeautyDFF.drop_duplicates()
BeautyDFF.shape

In [ ]:
sample_df = BeautyDFF.groupby(['userId_int']).agg({'productId_int':'count'}) 
sample_df = sample_df.rename(columns = {'productId_int':'Product_Count'}).reset_index() 
sample_ = sample_df[sample_df['Product_Count'] > 1]         #get list of users that have rated more than 5 products
print(sample_.shape[0])
onetime_userslist =  sample_['userId_int'].to_list()
#print(len(onetime_userslist))
#onetime_userslist[:10]

In [ ]:
# df_SAMPLE = BeautyDFF.groupby(['productId_int']).agg({'userId_int':'count'}) 
# df_SAMPLE = df_SAMPLE.rename(columns = {'userId_int':'User_Count'}).reset_index() 
# DF_Items = df_SAMPLE[df_SAMPLE['User_Count'] > 1]         #get list of users that have rated more than 5 products
# print(DF_Items.shape[0])
# onetime_itemslist =  DF_Items['productId_int'].to_list()
# print(len(onetime_itemslist))
# #onetime_itemslist[:10]

In [ ]:
DFF_ = BeautyDFF.loc[ BeautyDFF['userId_int'].isin(onetime_userslist) ]#& BeautyDFF['productId_int'].isin(onetime_itemslist)]
print(DFF_['productId_int'].nunique())
print(DFF_['userId_int'].nunique())

In [ ]:
#AMZB_1 = BeautyDFF.loc[BeautyDFF['userId_int'].isin(onetime_userslist) & BeautyDFF['productId_int'].isin(onetime_userslist)]
AMZB_1 = DFF_[['productId_int','userId_int','rating','timestamp']]
AMZB_1

In [ ]:
#AMZB_df = AMZB_11.rename(columns={'userId':'userId_int','productId':'productId_int'})
#AMZB_1['timestamp'] = pd.to_datetime(AMZB_1['timestamp']) 
AMZB_df = AMZB_1.copy()
print(AMZB_df.dtypes)
print()
print("nUnique users: ",AMZB_df['userId_int'].nunique())
print("nUnique items: ",AMZB_df['productId_int'].nunique())
AMZB_df = AMZB_df.sort_values(['timestamp'])
AMZB_df

In [ ]:
unique_USER_id = pd.unique(AMZB_df.userId_int)
unique_PRODUCT_id = pd.unique(AMZB_df.productId_int)

zipped_USER_id_pair = zip(unique_USER_id, range(0, len(unique_USER_id)+1, 1))
zipped_PRODUCT_id_pair = zip(unique_PRODUCT_id, range(0, len(unique_PRODUCT_id)+1, 1))

USER_id_dict = OrderedDict(zipped_USER_id_pair)
PRODUCT_id_dict = OrderedDict(zipped_PRODUCT_id_pair)

AMZB_df['userId'] = AMZB_df.userId_int.map(USER_id_dict)
AMZB_df['productId'] = AMZB_df.productId_int.map(PRODUCT_id_dict)
print(AMZB_df['userId'].max())
print(AMZB_df['productId'].max())
print()
print(AMZB_df['userId'].nunique())
print(AMZB_df['productId'].nunique())
AMZB_df

In [ ]:
AMZB_DF = AMZB_df[['userId','productId','rating','timestamp']]
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())
AMZB_DF

###Step 2: Import Clean Data 

In [6]:
AMZB_DF  = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Research/Test_Data/usedAMZB.csv')
AMZB_DF['timestamp'] = pd.to_datetime(AMZB_DF['timestamp']) 
print(AMZB_DF.shape)     #	productId_int	userId_int
print(AMZB_DF.dtypes)
AMZB_DF

(51869, 4)
userId                int64
productId             int64
rating                int64
timestamp    datetime64[ns]
dtype: object


,userId,productId,rating,timestamp
0,0,0,1,2001-06-08
1,1,0,1,2002-06-12
2,1,1,1,2002-06-12
3,2,1,1,2002-06-17
4,3,1,1,2002-07-04
...,...,...,...,...
51864,21470,6687,1,2018-09-21
51865,16839,5635,1,2018-09-21
51866,15319,8618,1,2018-09-25
51867,12925,5977,1,2018-09-25


In [7]:
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())

(51869, 4)
For users: 
22799
22798

For items: 
8620
8619

Timestamp
2001-06-08 00:00:00
2018-09-26 00:00:00


###Step 1: Data Splitting

In [8]:
pivotMonths_list  = getPivotMonths(AMZB_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-08-26 00:00:00'),
 Timestamp('2018-07-26 00:00:00'),
 Timestamp('2018-06-26 00:00:00'),
 Timestamp('2018-05-26 00:00:00'),
 Timestamp('2018-04-26 00:00:00'),
 Timestamp('2018-03-26 00:00:00'),
 Timestamp('2018-02-26 00:00:00'),
 Timestamp('2018-01-26 00:00:00'),
 Timestamp('2017-12-26 00:00:00'),
 Timestamp('2017-11-26 00:00:00'),
 Timestamp('2017-10-26 00:00:00'),
 Timestamp('2017-09-26 00:00:00'),
 Timestamp('2017-08-26 00:00:00'),
 Timestamp('2017-07-26 00:00:00'),
 Timestamp('2017-06-26 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-04-26 00:00:00'),
 Timestamp('2017-03-26 00:00:00')]

In [9]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZB_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-03-25 00:00:00
Num ΔA_train : 10


In [10]:
AllDF_list, PSITest_list,HOLDOUT_list = TestTrain_DataSplit(AMZB_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


### Step 2: Find new users and items

In [11]:
Tstps = AMZB_DF['timestamp']
tr_ts = ΔA_train[-1]['timestamp'].max()
print("ΔA_train_last max date: ",tr_ts)
AllDF_start = AMZB_DF.loc[Tstps <= tr_ts] 
print(AllDF_start['timestamp'].min())
print(AllDF_start['timestamp'].max())
print()
print(AllDF_list[0]['timestamp'].min())
print(AllDF_list[0]['timestamp'].max())

ΔA_train_last max date:  2018-01-25 00:00:00
2001-06-08 00:00:00
2018-01-25 00:00:00

2001-06-08 00:00:00
2018-02-25 00:00:00


In [12]:
New_usersList,New_itemsList = Find_NewUsersItems(AllDF_start,AllDF_list,'userId','productId',N_steps=8)
New_usersList[0]   #[:10]

array([22340, 22341, 22342, 22343, 22344, 22345, 22346, 22347, 22348,
       22349, 22350, 22351, 22352, 22353, 22354, 22355, 22356, 22357,
       22358, 22359, 22360, 22361, 22362, 22363, 22364, 22365, 22366,
       22367, 22368, 22369, 22370, 22371, 22372, 22373, 22374, 22375,
       22376, 22377, 22378, 22379, 22380, 22381, 22382, 22383, 22384,
       22385, 22386, 22387, 22388, 22389, 22390, 22391, 22392, 22393,
       22394, 22395, 22396, 22397, 22398, 22399, 22400, 22401, 22402,
       22403, 22404, 22405, 22406, 22407, 22408, 22409, 22410, 22411,
       22412, 22413, 22414, 22415, 22416, 22417, 22418, 22419, 22420,
       22421, 22422, 22423, 22424, 22425, 22426, 22427, 22428, 22429,
       22430, 22431, 22432, 22433, 22434, 22435, 22436, 22437, 22438,
       22439, 22440, 22441, 22442, 22443, 22444, 22445, 22446, 22447,
       22448, 22449, 22450])

In [ ]:
New_itemsList[0]    #[:10]

array([8328, 8329, 8330, 8331, 8332, 8333, 8334, 8335, 8336, 8337, 8338,
       8339, 8340, 8341, 8342, 8343, 8344, 8345, 8346, 8347, 8348, 8349,
       8350, 8351, 8352, 8353, 8354, 8355, 8356, 8357, 8358, 8359, 8360,
       8361, 8362, 8363, 8364, 8365, 8366, 8367, 8368, 8369, 8370, 8371,
       8372, 8373, 8374, 8375, 8376, 8377, 8378, 8379, 8380, 8381, 8382,
       8383, 8384, 8385, 8386, 8387, 8388, 8389, 8390, 8391, 8392, 8393,
       8394, 8395, 8396, 8397, 8398])

In [13]:
for i,j in zip(New_usersList,New_itemsList):
    print(i.size)
    print(j.size)
    print()

111
71

97
56

119
44

78
36

24
26

17
28

12
20

1
11



###Get Updt RatMat

In [ ]:
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())   #

22340 8328


In [14]:
rows_i = AllDF_start['userId'].nunique()
cols_i = AllDF_start['productId'].nunique()
print(A0_df.shape)
A0_RatMat_updt = SingleRatingMatrix(A0_df,'userId', 'productId',rows_i,cols_i)
A0_RatMat_updt

(45139, 4)


<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
	with 44740 stored elements in Compressed Sparse Row format>

In [15]:
PSI_TrainMat_updt = AllRatingMatrices(ΔA_train,'userId','productId',rows_i ,cols_i)
PSI_TrainMat_updt

[<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 553 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 518 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 432 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 397 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 1017 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 482 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 357 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse mat

In [ ]:
print(ΔA_train[-1]['userId'].max(),AllDF_start['userId'].nunique())
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique()) 

22339 22340
22340 8328


In [16]:
U_start,S_start,V_start = last_psiTrainMat(A0_RatMat_updt,PSI_TrainMat_updt,k=50)
print()
print("U_start shape: ",U_start.shape)
print("S_start shape: ",S_start.shape)
print("V_start shape: ",V_start.shape)

100%|██████████| 10/10 [00:01<00:00,  8.75it/s]


U_start shape:  (22340, 50)
S_start shape:  (50, 50)
V_start shape:  (8328, 50)


### Allsteps Update

In [17]:
U_list,S_list,V_list = [],[],[]
U_list.append(U_start)
S_list.append(S_start)
V_list.append(V_start)
print(len(U_list),len(V_list))

1 1


In [18]:
DItems_, DUsers_,In_DomainUSERS,In_DomainITEMS,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,U_list,S_list,V_list = ALLSTEPs_update(AllDF_start,
                                                                          AllDF_list,New_itemsList,New_usersList,U_list,S_list,V_list)

  1%|          | 1/111 [00:00<00:11,  9.38it/s]

1st Updating Phase..

Is_UserCold? : False
Is_ItemCold? : True
Is_ItemCold? : False
22340 8329

Is_UserCold? : False
Is_ItemCold? : False
22341 8330

Is_UserCold? : False


  3%|▎         | 3/111 [00:00<00:10, 10.10it/s]

Is_ItemCold? : False
22342 8331

Is_UserCold? : False
Is_ItemCold? : False
22343 8332

Is_UserCold? : False
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : False
22344 8336


  5%|▌         | 6/111 [00:00<00:11,  9.38it/s]

...

Is_ItemCold? : False

Is_ItemCold? : False

Is_ItemCold? : False

Is_ItemCold? : False

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True



100%|██████████| 58/58 [00:01<00:00, 56.87it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
2nd Updating Phase..

Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True



100%|██████████| 7/7 [00:29<00:00,  4.15s/it]

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True


In [19]:
print(len(In_DomainITEMS[8327+1:]))
print(In_DomainITEMS[8327+1:])
print()
print(len(In_DomainUSERS[22339+1:]))
print(In_DomainUSERS[22339+1:])

245
[8329, 8330, 8331, 8332, 8336, 8343, 8344, 8345, 8346, 8349, 8352, 8355, 8361, 8363, 8364, 8368, 8369, 8370, 8371, 8372, 8373, 8374, 8375, 8378, 8379, 8385, 8386, 8389, 8391, 8398, 8328, 8333, 8334, 8340, 8348, 8353, 8357, 8362, 8365, 8367, 8376, 8377, 8380, 8381, 8383, 8384, 8388, 8390, 8392, 8395, 8399, 8400, 8401, 8404, 8405, 8406, 8407, 8408, 8409, 8410, 8411, 8413, 8414, 8415, 8416, 8417, 8418, 8419, 8422, 8423, 8424, 8428, 8431, 8433, 8437, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8448, 8449, 8451, 8452, 8453, 8454, 8420, 8427, 8432, 8434, 8435, 8436, 8450, 8360, 8455, 8456, 8457, 8458, 8459, 8461, 8464, 8465, 8467, 8470, 8472, 8474, 8475, 8476, 8478, 8479, 8481, 8482, 8483, 8484, 8487, 8489, 8490, 8495, 8412, 8463, 8468, 8469, 8471, 8421, 8351, 8499, 8500, 8501, 8504, 8505, 8506, 8508, 8509, 8510, 8511, 8512, 8513, 8514, 8515, 8516, 8517, 8518, 8520, 8522, 8523, 8524, 8530, 8531, 8532, 8354, 8507, 8519, 8521, 8525, 8526, 8529, 8460, 8498, 8350, 8356, 8535,

In [ ]:
#deferred items and users
print(len(DItems_))
print(DItems_)
print()
print(len(DUsers_))
print(DUsers_)

47
[8566, 8567, 8570, 8572, 8573, 8585, 8586, 8546, 8547, 8502, 8503, 8527, 8528, 8533, 8534, 8466, 8473, 8477, 8480, 8485, 8486, 8488, 8491, 8492, 8493, 8494, 8496, 8497, 8402, 8403, 8425, 8426, 8429, 8430, 8335, 8338, 8339, 8341, 8342, 8347, 8358, 8359, 8382, 8393, 8394, 8396, 8397]

23
[22771, 22776, 22785, 22751, 22677, 22734, 22742, 22613, 22637, 22646, 22651, 22653, 22659, 22453, 22487, 22497, 22355, 22362, 22365, 22372, 22388, 22443, 22446]


In [ ]:
for x in list(itemID_dict)[8355:]:
    print ("Id: {}, UpdtId: {} ".format(x,  itemID_dict[x]))   ##id ==key || updtedid == values  ,itemID_dict

Id: 8389, UpdtId: 8355 
Id: 8391, UpdtId: 8356 
Id: 8398, UpdtId: 8357 
Id: 8328, UpdtId: 8358 
Id: 8333, UpdtId: 8359 
Id: 8334, UpdtId: 8360 
Id: 8340, UpdtId: 8361 
Id: 8348, UpdtId: 8362 
Id: 8353, UpdtId: 8363 
Id: 8357, UpdtId: 8364 
Id: 8362, UpdtId: 8365 
Id: 8365, UpdtId: 8366 
Id: 8367, UpdtId: 8367 
Id: 8376, UpdtId: 8368 
Id: 8377, UpdtId: 8369 
Id: 8380, UpdtId: 8370 
Id: 8381, UpdtId: 8371 
Id: 8383, UpdtId: 8372 
Id: 8384, UpdtId: 8373 
Id: 8388, UpdtId: 8374 
Id: 8390, UpdtId: 8375 
Id: 8392, UpdtId: 8376 
Id: 8395, UpdtId: 8377 
Id: 8399, UpdtId: 8378 
Id: 8400, UpdtId: 8379 
Id: 8401, UpdtId: 8380 
Id: 8404, UpdtId: 8381 
Id: 8405, UpdtId: 8382 
Id: 8406, UpdtId: 8383 
Id: 8407, UpdtId: 8384 
Id: 8408, UpdtId: 8385 
Id: 8409, UpdtId: 8386 
Id: 8410, UpdtId: 8387 
Id: 8411, UpdtId: 8388 
Id: 8413, UpdtId: 8389 
Id: 8414, UpdtId: 8390 
Id: 8415, UpdtId: 8391 
Id: 8416, UpdtId: 8392 
Id: 8417, UpdtId: 8393 
Id: 8418, UpdtId: 8394 
Id: 8419, UpdtId: 8395 
Id: 8422, UpdtId

In [ ]:
for x in list(userID_dict)[22339:]:
    print ("Id: {}, UpdtId: {} ".format(x,  userID_dict[x]))   ##id ==key || updtedid == values 

Id: 22339, UpdtId: 22339 
Id: 22340, UpdtId: 22340 
Id: 22341, UpdtId: 22341 
Id: 22342, UpdtId: 22342 
Id: 22343, UpdtId: 22343 
Id: 22344, UpdtId: 22344 
Id: 22345, UpdtId: 22345 
Id: 22346, UpdtId: 22346 
Id: 22347, UpdtId: 22347 
Id: 22348, UpdtId: 22348 
Id: 22349, UpdtId: 22349 
Id: 22350, UpdtId: 22350 
Id: 22351, UpdtId: 22351 
Id: 22352, UpdtId: 22352 
Id: 22353, UpdtId: 22353 
Id: 22354, UpdtId: 22354 
Id: 22356, UpdtId: 22355 
Id: 22357, UpdtId: 22356 
Id: 22358, UpdtId: 22357 
Id: 22359, UpdtId: 22358 
Id: 22361, UpdtId: 22359 
Id: 22363, UpdtId: 22360 
Id: 22364, UpdtId: 22361 
Id: 22366, UpdtId: 22362 
Id: 22367, UpdtId: 22363 
Id: 22368, UpdtId: 22364 
Id: 22369, UpdtId: 22365 
Id: 22370, UpdtId: 22366 
Id: 22371, UpdtId: 22367 
Id: 22373, UpdtId: 22368 
Id: 22374, UpdtId: 22369 
Id: 22375, UpdtId: 22370 
Id: 22376, UpdtId: 22371 
Id: 22378, UpdtId: 22372 
Id: 22379, UpdtId: 22373 
Id: 22380, UpdtId: 22374 
Id: 22381, UpdtId: 22375 
Id: 22382, UpdtId: 22376 
Id: 22383, U

In [ ]:
print(len(U_list),len(V_list))
for u,v in zip(U_list,V_list):
    print(u.shape,v.shape)   

9 9
(22340, 50) (8328, 50)
(22438, 50) (8378, 50)
(22532, 50) (8427, 50)
(22643, 50) (8458, 50)
(22722, 50) (8493, 50)
(22749, 50) (8521, 50)
(22763, 50) (8542, 50)
(22775, 50) (8562, 50)
(22776, 50) (8573, 50)


In [ ]:
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22438 8378
22532 8427
22643 8458
22722 8493
22749 8521
22763 8542
22775 8562
22776 8573


In [ ]:
print(len(In_DomainUSERS),len(In_DomainITEMS))

22776 8573


### Get_newHoldoutDF

In [20]:
newHOLDOUT_LIST = get_NEWHoldout(HOLDOUT_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
len(newHOLDOUT_LIST)

100%|██████████| 8/8 [00:00<00:00, 170.56it/s]


8

In [21]:
for old,new in zip(HOLDOUT_list,newHOLDOUT_LIST):
    print(old.shape[0],new.shape[0])

258 245
239 234
253 243
207 204
148 147
97 93
66 66
30 30


In [ ]:
newHOLDOUT_LIST[0]['Updated_UserID'].max()

22437

### Get UserItem MAT

In [ ]:
userID_dict

In [24]:
def adjustedAllDF(AllDF_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n):
    newAllDF_list =[]
    for i in range(n):
        AllDF_list[i] = AllDF_list[i][['userId','productId']]
        Updt_Items    = AllUpdtITEMS_[i]
        Updt_Users    = AllUpdtUSERS_[i]     
        allnew_df     = AllDF_list[i].loc[(AllDF_list[i]['productId'].isin(Updt_Items)) & (AllDF_list[i]['userId'].isin(Updt_Users))]

        prevUser_ID =   allnew_df['userId'].values  
        prevItems_ID =  allnew_df['productId'].values   
        Updted_UserID = [userID_dict.get(user) for user in prevUser_ID]   
        Updted_ItemID = [itemID_dict.get(item) for item in prevItems_ID]
        allnew_df['Updated_UserID'] = Updted_UserID
        allnew_df['Updated_ItemID'] = Updted_ItemID
        newAllDF_list.append(allnew_df)
    return newAllDF_list




In [26]:
newAllDF_list = adjustedAllDF(AllDF_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())
print()
for df in newAllDF_list:
    print(df['Updated_UserID'].nunique(),df['Updated_ItemID'].nunique())

22340 8328

22438 8378
22532 8427
22643 8458
22722 8493
22749 8521
22763 8542
22775 8562
22776 8573


In [29]:
UserItem_Mat = All_SingleStepRatMat(newAllDF_list,'Updated_UserID','Updated_ItemID')
print(len(UserItem_Mat))
UserItem_Mat

8


[<22438x8378 sparse matrix of type '<class 'numpy.float64'>'
 	with 49858 stored elements in Compressed Sparse Row format>,
 <22532x8427 sparse matrix of type '<class 'numpy.float64'>'
 	with 50186 stored elements in Compressed Sparse Row format>,
 <22643x8458 sparse matrix of type '<class 'numpy.float64'>'
 	with 50599 stored elements in Compressed Sparse Row format>,
 <22722x8493 sparse matrix of type '<class 'numpy.float64'>'
 	with 50919 stored elements in Compressed Sparse Row format>,
 <22749x8521 sparse matrix of type '<class 'numpy.float64'>'
 	with 51094 stored elements in Compressed Sparse Row format>,
 <22763x8542 sparse matrix of type '<class 'numpy.float64'>'
 	with 51202 stored elements in Compressed Sparse Row format>,
 <22775x8562 sparse matrix of type '<class 'numpy.float64'>'
 	with 51277 stored elements in Compressed Sparse Row format>,
 <22776x8573 sparse matrix of type '<class 'numpy.float64'>'
 	with 51310 stored elements in Compressed Sparse Row format>]

In [30]:
print(len(U_list),len(V_list))
for u,v in zip(U_list,V_list):
    print(u.shape[0],v.shape[0])   

9 9
22340 8328
22438 8378
22532 8427
22643 8458
22722 8493
22749 8521
22763 8542
22775 8562
22776 8573


In [31]:
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())
print()
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22340 8328

22438 8378
22532 8427
22643 8458
22722 8493
22749 8521
22763 8542
22775 8562
22776 8573


### PREDICTION

In [ ]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [ ]:
newHOLDOUT_LIST[0].columns

In [32]:
Vpsi_listUpdt = V_list[1:]
for v in Vpsi_listUpdt:
    print(v.shape)

(8378, 50)
(8427, 50)
(8458, 50)
(8493, 50)
(8521, 50)
(8542, 50)
(8562, 50)
(8573, 50)


In [33]:
All_PSI_UPDTPRED =  TQDMgetALLTopNPred(UserItem_Mat,newHOLDOUT_LIST,Vpsi_listUpdt,'Updated_UserID',N = 10)
print(len(All_PSI_UPDTPRED))
All_PSI_UPDTPRED[:2]

8it [00:00, 25.03it/s]

8


[array([[ 439,  717,  230, ..., 3985, 1230,  415],
        [ 922,  519,  194, ..., 3666,  231,  557],
        [  63,  520,   43, ...,  137,   36,  134],
        ...,
        [ 519,  381,   60, ...,  520, 1725,  694],
        [ 191,   43,  613, ...,  512,  566, 4714],
        [ 389,  133,  598, ..., 1034,   82, 1310]]),
 array([[1409,  566, 3727, ...,  457, 2543,  389],
        [  68,   69,  457, ..., 3139,   18, 3026],
        [1202,  899,  677, ..., 1409,  833, 1164],
        ...,
        [1765,   68,   69, ...,  362,  151, 1203],
        [ 389,  133,  598, ..., 1034,   82, 1310],
        [ 566,  606, 3442, ..., 4231,  516, 1549]])]

In [ ]:
All_PSI_UPDTPRED[0].shape

(245, 10)

In [34]:
AllSteps_HitR_updt, LowerBand_updt, Avg_HitR_updt, UpperBand_updt   = getAll_HitRate(newHOLDOUT_LIST,All_PSI_UPDTPRED,'Updated_UserID','Updated_ItemID')

Number of hits:  0
Total Num of users:  245
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  234
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  243
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  204
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  147
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  93
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  66
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  30
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.0


###  RandRec:Test

In [ ]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [35]:
All_RandPred = get_ALLRandPred(UserItem_Mat,newHOLDOUT_LIST,'Updated_UserID',N=10)
All_RandPred[:2]

8it [00:00, 28.95it/s]


[array([[7555, 7330, 3307, ...,  235, 6928, 4176],
        [6361, 7917,  615, ..., 3761, 2344, 5562],
        [5991, 4866, 4440, ...,  718, 2240, 5167],
        ...,
        [6465,  616, 1220, ..., 3243, 3409, 3850],
        [7337, 6236, 2489, ..., 2137, 3550, 5808],
        [6900, 1052, 4751, ..., 3136, 3478, 7238]]),
 array([[3840, 1832, 6094, ..., 2071, 6278, 5993],
        [7146, 7018,  950, ..., 8344, 5630, 4626],
        [2878, 8287, 2722, ..., 5747, 2791, 6784],
        ...,
        [2477, 1462, 6696, ..., 8240, 6521, 8091],
        [4319, 7401, 3868, ..., 2551, 3881, 6271],
        [ 531, 7235, 5840, ...,  252, 1319,  884]])]

In [ ]:
All_RandPred[0].shape

(245, 10)

In [36]:
 AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand  = getAll_RandomHitRate(newHOLDOUT_LIST,All_RandPred,'Updated_UserID','Updated_ItemID')

Number of hits:  0
Total Num of users:  245
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  234
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  243
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  204
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  147
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  93
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  66
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  30
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.0


### Most Pop Rec

In [ ]:
PSITest_list[1]['timestamp'].min()

In [37]:
def adjustedPSI_DF(PSITest_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n):
    new_PSIDFlist =[]
    for i in range(n):
        PSITest_list[i] = PSITest_list[i][['userId','productId']]
        Updt_Items    = AllUpdtITEMS_[i]
        Updt_Users    = AllUpdtUSERS_[i]     
        new_PSIdf     = PSITest_list[i].loc[(PSITest_list[i]['productId'].isin(Updt_Items)) & (PSITest_list[i]['userId'].isin(Updt_Users))]

        prevUser_ID =   new_PSIdf['userId'].values  
        prevItems_ID =  new_PSIdf['productId'].values   
        Updted_UserID = [userID_dict.get(user) for user in prevUser_ID]   
        Updted_ItemID = [itemID_dict.get(item) for item in prevItems_ID]
        new_PSIdf['Updated_UserID'] = Updted_UserID
        new_PSIdf['Updated_ItemID'] = Updted_ItemID
        new_PSIDFlist.append(new_PSIdf)
    return new_PSIDFlist


In [39]:
new_PSIDFlist = adjustedPSI_DF(PSITest_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
for old,new in zip(PSITest_list,new_PSIDFlist):
    print(old.shape[0],new.shape[0])

110 102
104 101
178 172
114 111
25 24
18 15
10 10
3 3


In [ ]:
new_PSIDFlist[0]

,userId,productId,Updated_UserID,Updated_ItemID
50009,22340,1514,22340,1514
50011,22341,413,22341,413
50014,22342,948,22342,948
50015,22343,8328,22343,8358
50025,22346,4078,22346,4078
...,...,...,...,...
50360,22445,251,22433,251
50365,22445,815,22433,815
50366,20754,105,20754,105
50367,22449,168,22436,168


In [ ]:
MostPOPPred_ = getMOSTPOP_Pred(new_PSIDFlist[0],'Updated_UserID','Updated_ItemID',Nusers=10,N=10)
MostPOPPred_

array([[ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369],
       [ 74, 106, 168, 239, 230, 555, 319, 251, 815, 369]])

In [41]:
All_MostPOPRED_List =  getAll_MOSTPOP_Pred(new_PSIDFlist,newHOLDOUT_LIST,'Updated_UserID','Updated_ItemID',N=10)
All_MostPOPRED_List[:2]

[array([[ 74, 106, 168, ..., 251, 815, 369],
        [ 74, 106, 168, ..., 251, 815, 369],
        [ 74, 106, 168, ..., 251, 815, 369],
        ...,
        [ 74, 106, 168, ..., 251, 815, 369],
        [ 74, 106, 168, ..., 251, 815, 369],
        [ 74, 106, 168, ..., 251, 815, 369]]),
 array([[110,  74, 376, ..., 251, 239, 948],
        [110,  74, 376, ..., 251, 239, 948],
        [110,  74, 376, ..., 251, 239, 948],
        ...,
        [110,  74, 376, ..., 251, 239, 948],
        [110,  74, 376, ..., 251, 239, 948],
        [110,  74, 376, ..., 251, 239, 948]])]

In [ ]:
All_MostPOPRED_List[0].shape

(245, 10)

In [42]:
AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand  = getAll_MostPOPHitRate(newHOLDOUT_LIST,All_MostPOPRED_List,'Updated_UserID','Updated_ItemID')

Number of hits:  24
Total Num of users:  245
Recommendation HitRate:  0.09795918367346938
Number of hits:  20
Total Num of users:  234
Recommendation HitRate:  0.08547008547008547
Number of hits:  57
Total Num of users:  243
Recommendation HitRate:  0.2345679012345679
Number of hits:  32
Total Num of users:  204
Recommendation HitRate:  0.1568627450980392
Number of hits:  24
Total Num of users:  147
Recommendation HitRate:  0.16326530612244897
Number of hits:  1
Total Num of users:  93
Recommendation HitRate:  0.010752688172043012
Number of hits:  4
Total Num of users:  66
Recommendation HitRate:  0.06060606060606061
Number of hits:  0
Total Num of users:  30
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.10118549629708933


In [43]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


## Apply Functions on *AMZGames*

### Import Cleaned Data

In [44]:
AMZGames_DF  = pd.read_csv('/content/usedAMZGames.csv')
AMZGames_DF['timestamp'] = pd.to_datetime(AMZGames_DF['timestamp']) 
print(AMZGames_DF.shape)       #productId_int	userId_int
print(AMZGames_DF.dtypes)
print()
AMZGames_DF

(372453, 6)
userId_int               object
productId_int            object
rating                    int64
timestamp        datetime64[ns]
userId                    int64
productId                 int64
dtype: object



,userId_int,productId_int,rating,timestamp,userId,productId
0,A3BEJOL2X8PS8I,B00004TC6E,1,2001-01-01,0,0
1,A1SQUI6BNEOB6C,B00004UE0G,1,2001-01-01,1,1
2,A2ZESFCRJL7YA0,B00002CF9M,1,2001-01-01,2,2
3,A3URVWPFJOHT72,B00000K1AF,1,2001-01-01,3,3
4,A1EE8EZLR8LJ9M,B00004WLMA,1,2001-01-02,4,4
...,...,...,...,...,...,...
372448,A2UVBD3N7449IC,B01ABTJ2BM,1,2018-08-31,28129,16648
372449,A27F1N27M1X0NM,B01D8H09LQ,1,2018-08-31,3183,16542
372450,A1J5HIF41ENSMZ,B01D8H09LQ,1,2018-08-31,16433,16542
372451,A18C19UTTAQEOD,B01GW3P9PE,1,2018-08-31,16400,17342


In [45]:
AMZGames_DF = AMZGames_DF[['userId','productId','rating','timestamp']]
print(AMZGames_DF.shape)
print("For users: ")
print(AMZGames_DF['userId'].nunique())
print(AMZGames_DF['userId'].max())
print()
print("For items: ")
print(AMZGames_DF['productId'].nunique())
print(AMZGames_DF['productId'].max())
print()
print("Timestamp")
print(AMZGames_DF['timestamp'].min())
print(AMZGames_DF['timestamp'].max())
AMZGames_DF

(372453, 4)
For users: 
53242
53241

For items: 
17351
17350

Timestamp
2001-01-01 00:00:00
2018-08-31 00:00:00


,userId,productId,rating,timestamp
0,0,0,1,2001-01-01
1,1,1,1,2001-01-01
2,2,2,1,2001-01-01
3,3,3,1,2001-01-01
4,4,4,1,2001-01-02
...,...,...,...,...
372448,28129,16648,1,2018-08-31
372449,3183,16542,1,2018-08-31
372450,16433,16542,1,2018-08-31
372451,16400,17342,1,2018-08-31


###Step 1: Data Splitting

In [46]:
pivotMonths_list  = getPivotMonths(AMZGames_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-07-31 00:00:00'),
 Timestamp('2018-06-30 00:00:00'),
 Timestamp('2018-05-31 00:00:00'),
 Timestamp('2018-04-30 00:00:00'),
 Timestamp('2018-03-31 00:00:00'),
 Timestamp('2018-02-28 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-09-30 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-02-28 00:00:00')]

In [47]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZGames_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-02-27 00:00:00
Num ΔA_train : 10


In [48]:
AllDF_list, PSITest_list,HOLDOUT_list = TestTrain_DataSplit(AMZGames_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [49]:
for df in ΔA_train:
    print(df['userId'].nunique(),df['productId'].nunique())

1959 2319
1455 1844
1236 1637
1080 1492
1033 1490
1041 1577
907 1355
871 1203
862 1228
845 1238


In [50]:
for df in ΔA_test:
    print(df['userId'].nunique(),df['productId'].nunique())

865 1224
687 1080
816 1462
718 1183
560 901
465 584
405 421
243 256


## Apply Functions on *AMZElect*

### Import Cleaned Data

In [52]:
!unzip  /content/usedAMZElect.zip

Archive:  /content/usedAMZElect.zip
  inflating: usedAMZElect.csv        


In [53]:
AMZElect_DF  = pd.read_csv('/content/usedAMZElect.csv')
AMZElect_DF['timestamp'] = pd.to_datetime(AMZElect_DF['timestamp']) 
print(AMZElect_DF.shape)      #productId_int	userId_int
print(AMZElect_DF.dtypes)
AMZElect_DF

(2760569, 4)
userId                int64
productId             int64
rating                int64
timestamp    datetime64[ns]
dtype: object


,userId,productId,rating,timestamp
0,0,0,1,2000-01-01
1,1,1,1,2000-01-02
2,2,2,1,2000-01-03
3,3,3,1,2000-01-06
4,4,4,1,2000-01-06
...,...,...,...,...
2760564,60863,220253,1,2018-08-31
2760565,9978,220253,1,2018-08-31
2760566,59037,238414,1,2018-08-31
2760567,75909,251125,1,2018-08-31


In [54]:
print(AMZElect_DF.shape)
print("For users: ")
print(AMZElect_DF['userId'].nunique())
print(AMZElect_DF['userId'].max())
print()
print("For items: ")
print(AMZElect_DF['productId'].nunique())
print(AMZElect_DF['productId'].max())
print()
print("Timestamp")
print(AMZElect_DF['timestamp'].min())
print(AMZElect_DF['timestamp'].max())
AMZElect_DF

(2760569, 4)
For users: 
161594
161593

For items: 
318020
318019

Timestamp
2000-01-01 00:00:00
2018-08-31 00:00:00


,userId,productId,rating,timestamp
0,0,0,1,2000-01-01
1,1,1,1,2000-01-02
2,2,2,1,2000-01-03
3,3,3,1,2000-01-06
4,4,4,1,2000-01-06
...,...,...,...,...
2760564,60863,220253,1,2018-08-31
2760565,9978,220253,1,2018-08-31
2760566,59037,238414,1,2018-08-31
2760567,75909,251125,1,2018-08-31


###Step 1: Data Splitting

In [55]:
pivotMonths_list  = getPivotMonths(AMZElect_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-07-31 00:00:00'),
 Timestamp('2018-06-30 00:00:00'),
 Timestamp('2018-05-31 00:00:00'),
 Timestamp('2018-04-30 00:00:00'),
 Timestamp('2018-03-31 00:00:00'),
 Timestamp('2018-02-28 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-09-30 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-02-28 00:00:00')]

In [58]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZElect_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-02-27 00:00:00
Num ΔA_train : 10


In [59]:
AllDF_list, PSITest_list,HOLDOUT_list = TestTrain_DataSplit(AMZElect_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [60]:
for df in ΔA_train:
    print(df['userId'].nunique(),df['productId'].nunique())

17570 21425
15010 19188
13933 17058
12711 16008
12667 15464
12695 15398
11746 14523
10733 13206
10129 12593
10375 12375


In [61]:
for df in ΔA_test:
    print(df['userId'].nunique(),df['productId'].nunique())

11572 13325
9614 11772
10842 13592
9888 12416
8939 10901
7677 8770
6609 6753
4279 3895


In [62]:
print(A0_df['productId'].max(),A0_df['productId'].nunique())
print(A0_df['userId'].max(),A0_df['userId'].nunique())

293278 293279
158466 158467
